# Single cosine distance classifier vs Ensemble of 4 classifiers 

### Load data and packages

In [2]:
import sys

sys.path.insert(1, '../')

from Utils import *
from modeling import *

# create_dir('output')
# 
seed = 962
np.random.seed(seed)

test_size = .1
X, y = load_data(n=200, std=.3, plot = False, save = False)

n_shots = 8192
n_run = 5

### Split data in training and test

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed, test_size=test_size)

Y_vector_train = label_to_array(y_train)
Y_vector_test = label_to_array(y_test)

print("Size Training Set: ", len(X_train))
print("Size Test Set: ", len(X_test))

Size Training Set:  180
Size Test Set:  20


### Quantum cosine classifier for all the observations of the test set

In [6]:
accuracy = []
brier_score = []

for i in range(n_run):
    
    n = range(len(X_train))

    predictions = []

    # np.random.seed(seed)
    for x_test, y_ts in zip(X_test, Y_vector_test):
        ix = np.random.choice(n, 1)[0]
        x_train = X_train[ix]
        x_tr = normalize_custom(x_train)
        y_tr = Y_vector_train[ix]
        x_ts = normalize_custom(x_test)

        qc = cos_classifier(x_tr, x_ts, y_tr)
        r = exec_simulator(qc, n_shots=n_shots)

        predictions.append(retrieve_proba(r))

    a, b = evaluation_metrics(predictions, X_test, y_test)
    
    accuracy.append(a)
    brier_score.append(b)

print('Accuracy mean:', np.mean(accuracy))
print('Brier score mean:', np.mean(brier_score))

Accuracy mean: 0.5900000000000001
Brier score mean: 0.19787142947316172


### Choose device

In [7]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')
provider.backends()
backend = provider.get_backend('ibmq_qasm_simulator')

In [8]:
d=1
accuracy = []
brier_score = []

for i in range(n_run):
    
    predictions = []
    # np.random.seed(seed)

    for x_test, y_ts in zip(X_test, Y_vector_test):
        X_data, Y_data = training_set(X_train, y_train, n=2**d, seed=seed)
        x_test = normalize_custom(x_test)

        qc = ensemble(X_data, Y_data, x_test, n_swap=1, d=d, balanced=True)
        qc = transpile(qc, basis_gates = ['u1', 'u2', 'u3', 'cx'], optimization_level=3)
        # r = exec_simulator(qc, n_shots=n_shots)

        job = execute(qc, backend, shots = n_shots)
        results = job.result()
        r = results.get_counts(qc)

        predictions.append(retrieve_proba(r))

    a, b = evaluation_metrics(predictions, X_test, y_test)
        
    accuracy.append(a)
    brier_score.append(b)

print('Accuracy mean:', np.mean(accuracy))
print('Brier score mean:', np.mean(brier_score))

Accuracy mean: 0.75
Brier score mean: 0.17782156422734263


In [ ]:
d=2
accuracy = []
brier_score = []

for i in range(n_run):
    
    predictions = []

    #np.random.seed(seed)
    for x_test, y_ts in zip(X_test, Y_vector_test):
        X_data, Y_data = training_set(X_train, y_train, n=2**d, seed=seed)
        x_test = normalize_custom(x_test)

        qc = ensemble(X_data, Y_data, x_test, n_swap=1, d=d, balanced=True)
        qc = transpile(qc, basis_gates = ['u1', 'u2', 'u3', 'cx'], optimization_level=3)
        # r = exec_simulator(qc, n_shots=n_shots)

        job = execute(qc, backend, shots = n_shots)
        results = job.result()
        r = results.get_counts(qc)

        predictions.append(retrieve_proba(r))

    a, b = evaluation_metrics(predictions, X_test, y_test)
        
    accuracy.append(a)
    brier_score.append(b)

print('Accuracy mean:', np.mean(accuracy))
print('Brier score mean:', np.mean(brier_score))

In [9]:
d=3
accuracy = []
brier_score = []

for i in range(n_run):

    predictions = []

    # np.random.seed(seed)
    for x_test, y_ts in zip(X_test, Y_vector_test):
        X_data, Y_data = training_set(X_train, y_train, n=2**d, seed=seed)
        x_test = normalize_custom(x_test)

        qc = ensemble(X_data, Y_data, x_test, n_swap=1, d=d, balanced=True)
        qc = transpile(qc, basis_gates = ['u1', 'u2', 'u3', 'cx'], optimization_level=3)
        # r = exec_simulator(qc, n_shots=n_shots)

        job = execute(qc, backend, shots = n_shots)
        results = job.result()
        r = results.get_counts(qc)

        predictions.append(retrieve_proba(r))
        
    a, b = evaluation_metrics(predictions, X_test, y_test)
        
    accuracy.append(a)
    brier_score.append(b)

print('Accuracy mean:', np.mean(accuracy))
print('Brier score mean:', np.mean(brier_score))

IBMQBackendApiError: 'Error submitting job: \'(\\\'Connection aborted.\\\', OSError("(10060, \\\'WSAETIMEDOUT\\\')"))\''

In [ ]:
d=4

predictions = []

np.random.seed(seed)

for x_test, y_ts in zip(X_test, Y_vector_test):
    X_data, Y_data = training_set(X_train, y_train, n=8, seed=seed)
    x_test = normalize_custom(x_test)
    
    qc = ensemble(X_data, Y_data, x_test, n_swap=1, d=d, balanced=True)
    qc = transpile(qc, basis_gates = ['u1', 'u2', 'u3', 'cx'], optimization_level=3)
    # r = exec_simulator(qc, n_shots=n_shots)

    job = execute(qc, backend, shots = n_shots)
    results = job.result()
    r = results.get_counts(qc)
    
    predictions.append(retrieve_proba(r))
        
acc, brier = evaluation_metrics(predictions, X_test, y_test)

print('Accuracy:', acc)
print('Brier Score:', brier)

In [10]:
accuracy

[]

In [ ]:
# IBMQ.load_account()
# provider = IBMQ.get_provider(hub='ibm-q')
# provider.backends()
# backend = provider.get_backend('ibmq_16_melbourne')